In [10]:

import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report





df = pd.read_csv("Fertilizer Prediction.csv")


le_soil = LabelEncoder()
le_crop = LabelEncoder()
le_fert = LabelEncoder()

df['Soil Type'] = le_soil.fit_transform(df['Soil Type'])
df['Crop Type'] = le_crop.fit_transform(df['Crop Type'])
df['Fertilizer Name'] = le_fert.fit_transform(df['Fertilizer Name'])





In [11]:
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'KNN': KNeighborsClassifier(),
    'SVM': SVC()
}

In [12]:

X = df.drop('Fertilizer Name', axis=1)
y = df['Fertilizer Name']


scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [13]:

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


from sklearn.svm import SVC

svm_model = SVC(random_state=42) 
svm_model.fit(X_train, y_train)


SVC(random_state=42)

In [14]:
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"SVM Accuracy: {accuracy:.2f}")

SVM Accuracy: 0.85


In [15]:
def fertilizer_recommendationSVM(temp, humidity, moisture, soil_type, crop_type, N, P, K):
  
    soil_type = soil_type.strip().title()
    crop_type = crop_type.strip().title()

    
    if soil_type not in le_soil.classes_:
        raise ValueError(f"Soil type '{soil_type}' is invalid. Choose from: {list(le_soil.classes_)}")
    if crop_type not in le_crop.classes_:
        raise ValueError(f"Crop type '{crop_type}' is invalid. Choose from: {list(le_crop.classes_)}")

    soil_code = le_soil.transform([soil_type])[0]
    crop_code = le_crop.transform([crop_type])[0]

    
    features = np.array([[temp, humidity, moisture, soil_code, crop_code, N, K, P]])
    scaled = scaler.transform(features)

   
    prediction = svm_model.predict(scaled)
    return le_fert.inverse_transform(prediction)[0]


In [16]:
fertilizer = fertilizer_recommendationSVM(
    temp=30,
    humidity=60,
    moisture=90,
    soil_type='Sandy',
    crop_type='cotton',
    N=25,
    P=15,
    K=10
)
print("Recommended Fertilizer:", fertilizer)


Recommended Fertilizer: 17-17-17


D:\Anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [17]:

rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train, y_train)

y_pred = rfc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Accuracy: {accuracy:.2f}")


Random Forest Accuracy: 0.95


In [18]:
best_model = None
best_accuracy = 0

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"\n{name} Accuracy: {acc:.2f}")
    print(classification_report(y_test, y_pred))

    if acc > best_accuracy:
        best_accuracy = acc
        best_model = model

print("\nBest model selected:", best_model)


Random Forest Accuracy: 0.95
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       1.00      1.00      1.00         1
           2       0.00      0.00      0.00         0
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         5
           5       1.00      1.00      1.00         5
           6       1.00      1.00      1.00         6

    accuracy                           0.95        20
   macro avg       0.86      0.79      0.81        20
weighted avg       1.00      0.95      0.97        20


KNN Accuracy: 0.70
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       0.33      1.00      0.50         1
           3       0.00      0.00      0.00         1
           4       0.60      0.60      0.60         5
           5       0.60      0.60      0.60         5
           6       1.00     

D:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Anaconda\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precisio

In [19]:
def fertilizer_recommendation(temp, humidity, moisture, soil_type, crop_type, N, P, K):
    
    soil_type = soil_type.strip().title()
    crop_type = crop_type.strip().title()

    
    if soil_type not in le_soil.classes_:
        raise ValueError(f"Soil type '{soil_type}' is invalid. Choose from: {list(le_soil.classes_)}")
    if crop_type not in le_crop.classes_:
        raise ValueError(f"Crop type '{crop_type}' is invalid. Choose from: {list(le_crop.classes_)}")

 
    soil_code = le_soil.transform([soil_type])[0]
    crop_code = le_crop.transform([crop_type])[0]

   
    features = np.array([[temp, humidity, moisture, soil_code, crop_code, N, K, P]])
    scaled = scaler.transform(features)

    
    prediction = rfc.predict(scaled)
    return le_fert.inverse_transform(prediction)[0]


In [20]:
fertilizer = fertilizer_recommendation(
    temp=30,
    humidity=60,
    moisture=90,
    soil_type='Sandy',
    crop_type='cotton',
    N=25,
    P=15,
    K=10
)
print("Recommended Fertilizer:", fertilizer)


Recommended Fertilizer: 17-17-17


D:\Anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [21]:
valid_soil_types = le_soil.classes_
print(valid_soil_types)

['Black' 'Clayey' 'Loamy' 'Red' 'Sandy']


In [22]:
fertilizer = fertilizer_recommendation(
    temp=22,
    humidity=70,
    moisture=35,
    soil_type='Black',
    crop_type='Barley',
    N=40,
    P=20,
    K=15
)
print("Recommended Fertilizer:", fertilizer)


Recommended Fertilizer: Urea


D:\Anaconda\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [23]:
import pickle
pickle.dump(rfc,open('fertilizer_model.pkl','wb'))
pickle.dump(scaler,open('fertilizer_encoder.pkl','wb'))

In [24]:
pickle.dump(scaler, open('fertilizer_scaler.pkl', 'wb'))


In [25]:
pickle.dump(le_soil, open('le_soil.pkl', 'wb'))
pickle.dump(le_crop, open('le_crop.pkl', 'wb'))
pickle.dump(le_fert, open('le_fert.pkl', 'wb'))
